El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [2]:
# Importar bibliotecas
import pandas as pd
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
# Cargar DataSet
df = pd.read_csv("datasets/car_data.csv")

In [5]:
# Exploracion inicial 
display(HTML('<h1> Rusty Bargain DataFrame'))
display(HTML('<hr>'))

display(HTML('<h2> Informacion general'))
display(df.info())
display(HTML('<hr>'))

display(HTML('<h2> Estadisticas descriptivas'))
display(df.describe())
display(HTML('<hr>'))

display(HTML('<h2> Muestra de tabla'))
display(df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
308259,02/04/2016 07:58,0,sedan,2005,auto,0,NaN,150000,12,NaN,mercedes_benz,NaN,02/04/2016 00:00,0,30823,06/04/2016 07:15
337606,02/04/2016 12:25,950,bus,2001,manual,125,zafira,150000,11,petrol,opel,NaN,02/04/2016 00:00,0,35510,05/04/2016 18:46
47088,19/03/2016 14:51,1999,wagon,2003,manual,121,ptcruiser,150000,9,gasoline,chrysler,no,19/03/2016 00:00,0,53844,06/04/2016 22:44
67647,10/03/2016 22:56,12500,sedan,2007,auto,250,NaN,150000,7,gasoline,bmw,no,10/03/2016 00:00,0,31515,05/04/2016 21:16
187732,20/03/2016 12:41,330,small,2000,manual,0,fiesta,125000,0,petrol,ford,NaN,20/03/2016 00:00,0,74076,20/03/2016 12:41


### Preprocesamiento de datos

In [10]:
# Eliminar filas duplicadas
print(f"Numero de duplicados previos: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"Verificacion de duplicados: {df.duplicated().sum()}")

Numero de duplicados previos: 0
Verificacion de duplicados: 0


### Preparacion de datos (Explicacion)

Durante este proceso de preparacion de datos se realizaron varias cosas, por ejemplo
- Se cargo el archivo car_data.csv en un dataframe y realizamos un analisis de su estructura
- Identificamos las columnas con valores nulos, cuantificando los porcentajes para determinar la mejor estrategia de manejo.

Preparacion de datos

- Se eliminaron las filas con valores nulos en VehicleType, FuelType, Gearbox y Model.
- Para la columna NotRepaired, imputamos los valores nulos con una categoría "desconocido" para mantener la integridad de los datos.
- Verificamos si quedaban valores nulos en otras columnas tras la imputación.

## Entrenamiento del modelo 

In [21]:

df_clean = filtered_df.copy()
categorical_columns = df_clean.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df_clean.select_dtypes(exclude=['object']).columns.tolist()
numerical_columns.remove('Price')

In [22]:

# Preprocesador para variables categóricas usando One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocesador para variables numéricas usando escalado estándar
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combinar los transformadores en un único preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df_clean.drop('Price', axis=1)
y = df_clean['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar el tamaño de los conjuntos de datos
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


((227300, 15), (56826, 15), (227300,), (56826,))

## Análisis del modelo

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos